In [5]:
# Import the needed Packages
# FMU Simulation
import MoBASimulator as mb
# Numpy
import numpy as np
# Bokeh for Plotting
import bokeh.plotting as bk
import bokeh.io as bi
#from bokeh.io import export_svgs
bi.output_notebook()
# Algorithms
import Algorithms as alg

# TUBS Rot, Gelb, Orange, Hellgrün, Grün, Dunkelgrün, Hellblau, Blau, Dunkelblau, Violett
TUBScolorscale=[(190,30,60,0.9),
               (255,200,42,0.9),
               (225,109,0,0.9),
               (172,193,58,0.9),
               (109,131,0,0.9),
               (0,83,74,0.9),
               (102,180,211,0.9),
               (0,112,155,0.9),
               (0,63,87,0.9),
               (138,48,127,0.9)]

# Model Parameter
k = [[5.,1.],[-5.,5]]
t = [[100.,10.],[10.,100.]]
l = [[40,4],[4,40]]
# Rosenbrock
#k = [[1.,2./3.],[1.,1.]]
#t = [[1.,1./3.],[1.,1.]]
#l = [[1e-10,1e-10],[1e-10,1e-10]]

# Woodberry
#k = [[12.8,-18.9],[6.6,-19.4]]
#t = [[16.7,21.0],[10.9,14.4]]
#l = [[1,3],[7,3]]


# The needed Parameter
K = np.zeros((2,2))
T = np.zeros((2,2))
L = np.zeros((2,2))

# Load a Model
sim = mb.Simulator()
sim.clear()
sim.loadModel("C:/Users/juliu/Documents/Thesis/Modelica/FMU/2_2/Masterthesis_Models_mimo_0processmodel.fmu")

# Show log window
sim.showLogWindow()

# Parameter Values
params = {}
# Loop over system size
for outputs in range(1,3):
    for inputs in range(1,3):
        # Process Parameter 
        # System Gain
        params.update({"fmu.num["+str(outputs)+","+str(inputs)+",1]": k[outputs-1][inputs-1]})
        # System Lag
        params.update({"fmu.den["+str(outputs)+","+str(inputs)+",1]": t[outputs-1][inputs-1]})
        params.update({"fmu.den["+str(outputs)+","+str(inputs)+",2]": 1})
        # System Delay
        params.update({"fmu.delay["+str(outputs)+","+str(inputs)+"]": l[outputs-1][inputs-1]})

# Set Parameter and show for checking
sim.set(params)
#sim.showParameterDialog()
#print(params)



# First run, Input 1 -> Output 1 & 2
sim.set({"fmu.u[1]": 1,"fmu.u[2]": 0})
# Set timestep = 1e-2, endtime = 100
sim.resetModelState()
res = sim.simulate(0.01, 500)

# Get the signals
y = res["fmu.y[1]"]
y2 = res["fmu.y[2]"]
u = res["fmu.u[1]"]
time = res["time"]

# Get TF from Input 1 to Output 1
K[0][0],T[0][0],L[0][0]=alg.Integral_Identification(y,u,time)
# Get TF from Input 1 to Output 2
K[1][0],T[1][0],L[1][0]=alg.Integral_Identification(y2,u,time)


# Second run, Input 2 -> Output 1 & 2
# Reload the Model to set everything to zero
sim.resetModelState()
sim.set({"fmu.u[1]":0, "fmu.u[2]":1})
# Set timestep = 1e-2, endtime = 100
res=sim.simulate(0.01, 500)

# Get the signals
y = res["fmu.y[1]"]
y2 = res["fmu.y[2]"]
u = res["fmu.u[2]"]
time = res["time"]

# Get TF from Input 2 to Output 1
K[0][1],T[0][1],L[0][1] = alg.Integral_Identification(y,u,time)
# Get TF from Input 2 to Output 2
K[1][1],T[1][1],L[1][1] = alg.Integral_Identification(y2,u,time)

# Make variable storage
y = []
u = []
time = []

# Loop over the methods for controller design
for methods in range(0,3):
    if methods == 0:
        # Make a decentralized controller
        KY,B,D = alg.Control_Decentral(K,T,L)
    elif methods == 1:
        # Make a decoupled controller with Astrom
        KY,B,D = alg.Control_Astrom(K,T,L,100*np.eye(2,2))
    else:
        # Make a decoupled controller with modified Astrom
        KY,B,D = alg.Control_Decoupled(K,T,L,100*np.eye(2,2))

    # Make zeros to 1e-10 for numerical stable process
    KY[KY==0] = 1e-10
    B[B==0] = 1e-10
    D[D==0] = 1e-10


    # Load the closed loop model
    sim.clear()
    sim.loadModel("C:/Users/juliu/Documents/Thesis/Modelica/FMU/2_2/Masterthesis_Models_mimo_0closedloop.fmu")
    # Parameter Valuess
    params = {}
    
    # Loop over system size
    for outputs in range(1,3):
        for inputs in range(1,3):
            # Controller Parameter 
            # Proportional Gain
            params.update({"fmu.kp["+str(outputs)+","+str(inputs)+"]": KY[outputs-1][inputs-1][0]})
             # Integral Gain
            params.update({"fmu.ki["+str(outputs)+","+str(inputs)+"]": KY[outputs-1][inputs-1][1]})
            # Decoupler
            params.update({"fmu.d["+str(outputs)+","+str(inputs)+"]": D[outputs-1][inputs-1]})
            # Set Point Weight
            params.update({"fmu.b["+str(outputs)+","+str(inputs)+"]": B[outputs-1][inputs-1]})
            # Process Parameter 
            # Process Parameter 
            # System Gain
            params.update({"fmu.num["+str(outputs)+","+str(inputs)+",1]": k[outputs-1][inputs-1]})
            # System Lag
            params.update({"fmu.den["+str(outputs)+","+str(inputs)+",1]": t[outputs-1][inputs-1]})
            params.update({"fmu.den["+str(outputs)+","+str(inputs)+",2]": 1})
            # System Delay
            params.update({"fmu.delay["+str(outputs)+","+str(inputs)+"]": l[outputs-1][inputs-1]})

    # Set Parameter Values
    sim.set(params)
    
    # First run, Input 1 -> Output 1 & 2
    sim.resetModelState()
    sim.set({"fmu.u[1]": 1.,"fmu.u[2]": 0.})
    res=sim.simulate(0.1, 1000)
    # Second run, Input 2-> Output 1 & 2
    sim.set({"fmu.u[1]": 1.,"fmu.u[2]": 1})
    res=sim.simulate(0.1, 2000)

    # Store the output
    y.append([res["fmu.y[1]"],res["fmu.y[2]"]])
    u.append([res["fmu.u[1]"],res["fmu.u[2]"]])
    time.append([res["time"],res["time"]])

Loading BokehJS ...

0.0155753146863
Aström Detuning Iterations:0
0.0155753146863
Aström Detuning Iterations:0
[[ 0.          0.18112434]
 [ 0.90562172  0.        ]]
0.0125458801384
(0.18112434437362993, 100.0)
Modified Detuning Iterationts 0
0.0125458801384
(0.9056217218681496, 100.0)
Modified Detuning Iterationts 0


In [6]:
from bokeh.layouts import gridplot,layout

p1 = bk.figure(title = "Process Example McAvory et.al.", height= 350, width = 800, y_range = (-0.2,1.5),  y_axis_label = "Output 1")
p1.line(time[0][0],y[0][0], color = TUBScolorscale[6], line_width = 2)
p1.line(time[1][0],y[1][0], color = TUBScolorscale[0], line_width = 2)
p1.line(time[2][0],y[2][0], color = TUBScolorscale[2], line_width = 2)


p2 = bk.figure( height= 300, width = 800, y_range = (-0.5, 1.2), x_axis_label = "Time [s]", y_axis_label = "Output 2")
p2.line(time[0][1],y[0][1], color = TUBScolorscale[6], line_width = 2, legend="RGA")
p2.line(time[1][1],y[1][1], color = TUBScolorscale[0], line_width = 2, legend="Aström Decoupling")
p2.line(time[2][1],y[2][1], color = TUBScolorscale[2], line_width = 2, legend="Modified Aström Decoupling")

p2.legend.location = "bottom_right"

p1.output_backend = "svg"
p2.output_backend = "svg"

p = layout([[p1],[p2]])
bk.show(p)